In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Algorithme de decision tree

1. Choix de la variable racine
celle qui permet de mieux séparer les données
- teste chaque variable seule
- sélectionne celle qui donne des feuilles avec la plus faible impureté

**Feuille pure** ne contient que des 1 ou que des 0
**Feuille impure** contient un mélange de 1 et de 0

**Gini Impurity d’une feuille** = $
1 - P(\text{oui})^2 - P(\text{non})^2
$

In [31]:
df =    pd.read_csv('../data/processed/train_optimized_v3.csv')
y = df['target']
X = df.drop(["target", "id"], axis=1)

In [60]:
X.head(1)

,keyword,text_cleaned,text_length,word_count,char_count,has_emergency_word,emergency_word_count,emergency_density,has_url,url_count,has_mention,mention_count,exclamation_count,intense_punctuation,avg_word_length,urgency_score,stopword_ratio,keyword_in_text
0,weapon,mention_token changing my weapon!,33,4,33,False,0,0.0,False,0,True,1,1,0,7.5,1.5,0.25,True


In [72]:
df.head()

,id,keyword,target,text_cleaned,text_length,word_count,char_count,has_emergency_word,emergency_word_count,emergency_density,has_url,url_count,has_mention,mention_count,exclamation_count,intense_punctuation,avg_word_length,urgency_score,stopword_ratio,keyword_in_text
0,10293,weapon,0,mention_token changing my weapon!,33,4,33,False,0,0.000000,False,0,True,1,1,0,7.500000,1.5,0.250000,True
1,5096,famine,1,url_token russian food crematoria provoke outr...,115,11,87,True,1,0.090909,True,2,False,0,0,0,7.000000,0.5,0.000000,True
2,4041,disaster,0,the mention_token quote website disaster. trie...,143,22,153,True,1,0.045455,False,0,True,1,0,0,6.000000,0.0,0.181818,True
3,1637,bombing,1,mention_token mention_token mention_token cong...,140,19,151,True,1,0.052632,False,0,True,2,0,0,7.000000,0.0,0.263158,True
4,7614,pandemonium,0,world class tgirl ass small_num - scene small_...,71,11,73,False,0,0.000000,True,1,False,0,0,0,5.727273,0.0,0.000000,True


### Lister les gini impurity de chaque variable 
PAS UTILE de créer un fichier<br>
un dictionnaire en mémoire suffit


In [86]:
gini_scores = {}

def gini(group):
    n = len(group)
    if n == 0:
        return 0
    p_counts = group.value_counts(normalize=True)
    return 1 - sum(p_counts**2)

### Variables booléennes

In [87]:
def bool_lower_impurity(df, col, target='target'):

    # Sépare les True et les False et récupère les valeurs de la cible
    true_group = df[df[col]]['target']
    false_group = df[~df[col]]['target']

    total = len(df)


    true_group_gini = gini(true_group)
    false_group_gini = gini(false_group)

    # Gini impurity total pondéré (utile pour le split global si tu veux)
    weighted_gini = (
        len(true_group) / total * true_group_gini +
        len(false_group) / total * false_group_gini
    )

    return true_group_gini, false_group_gini, weighted_gini


In [84]:
for col in df.select_dtypes(include='bool'):
        gini_true, gini_false, gini_weighted = bool_lower_impurity(df, col, target='target')
        print(f"{col}\ngini(True) = {gini_true}, gini(False) = {gini_false}, gini(weighted) = {gini_weighted}\n")
        gini_scores[col] = {
        'gini_true': gini_true,
        'gini_false': gini_false,
        'gini_weighted': gini_weighted
    }

has_emergency_word
gini(True) = 0.47978446495981275, gini(False) = 0.4262857682814558, gini(weighted) = 0.44540284136239217

has_url
gini(True) = 0.4982698961937716, gini(False) = 0.41934291857027683, gini(weighted) = 0.45820646636152235

has_mention
gini(True) = 0.44584924071088883, gini(False) = 0.4932802098833111, gini(weighted) = 0.4801568068269632

keyword_in_text
gini(True) = 0.47680473372781074, gini(False) = 0.4992154906431142, gini(weighted) = 0.4819654121085311



### Variables numérique

In [ ]:
def numeric_lower_impurity(df, col, target='target'):
    
    total = len(df)
    zeros = 0
    ones = 0

    for _ in df[col].unique():
        if df['target']==0:
            zeros +=1
        else:
            ones +=1


### Variables texte
Il faut les convertir en variables numériques : BagOfWords, TF_IDF, Word2Vec, ...

In [ ]:
def numeric_lower_impurity(col):
  pass

for col in df.select_dtypes(include='object'):
    # Appeler une fonction pour les colonnes texte